In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

API_TOKEN = "KuQ4Rt1ypOCvXfcm3cZXdPhOUlbuOrpHBgJkFm1MWTvtRR8TLhgEI02hjDxz"
BASE_URL = "https://api.sportmonks.com/v3/football"

# Fixture metadata (NOT relying on embedded odds for lines)
FIXTURES_INCLUDE = "participants;league.country"

# Odds endpoint includes (so bookmaker/market come through on the odds objects when supported)
ODDS_INCLUDE = "market;bookmaker"


def _safe_int(x):
    try:
        return int(x)
    except Exception:
        return None


def _extract_line_from_odd(odd: dict):
    """
    SportMonks odds endpoints often provide:
      - total (for goal line / O-U style markets)
      - handicap (for handicap markets)
    Sometimes also line.
    """
    for key in ("total", "handicap", "line"):
        v = odd.get(key)
        if v is None:
            continue
        try:
            return float(v)
        except Exception:
            # Sometimes it's non-numeric (rare); return as-is
            return v
    return None


def _extract_player_ids(odd: dict):
    """
    API Coach: odd['participants'] contains player id(s).
    Could be:
      - "12345"
      - 12345
      - [12345, 67890]
      - "12345,67890"
    We'll normalize to list[int].
    """
    p = odd.get("participants")
    if p is None:
        return []

    if isinstance(p, list):
        out = []
        for item in p:
            i = _safe_int(item)
            if i is not None:
                out.append(i)
        return out

    if isinstance(p, (int, float)):
        i = _safe_int(p)
        return [i] if i is not None else []

    if isinstance(p, str):
        # split on common separators
        parts = [s.strip() for s in p.replace(";", ",").split(",") if s.strip()]
        out = []
        for s in parts:
            i = _safe_int(s)
            if i is not None:
                out.append(i)
        return out

    return []


def fetch_player_name(player_id: int, session: requests.Session, cache: dict[int, str]) -> str | None:
    """
    Optional: resolve player id -> name via /players/{id}.
    Cached to avoid repeated calls.
    """
    if player_id in cache:
        return cache[player_id]

    try:
        r = session.get(
            f"{BASE_URL}/players/{player_id}",
            params={"api_token": API_TOKEN},
            timeout=30,
        )
        if r.status_code == 404:
            cache[player_id] = None
            return None
        r.raise_for_status()
        payload = r.json()
        name = (payload.get("data") or {}).get("name")
        cache[player_id] = name
        return name
    except Exception:
        cache[player_id] = None
        return None


def fetch_fixtures_next_12h() -> pd.DataFrame:
    now_utc = datetime.now(timezone.utc)
    end_utc = now_utc + timedelta(hours=12)

    start_date = now_utc.date()
    end_date = (now_utc + timedelta(days=1)).date()
    endpoint = f"/fixtures/between/{start_date:%Y-%m-%d}/{end_date:%Y-%m-%d}"

    session = requests.Session()
    fixtures = []
    page = 1

    while True:
        resp = session.get(
            f"{BASE_URL}{endpoint}",
            params={"api_token": API_TOKEN, "include": FIXTURES_INCLUDE, "page": page},
            timeout=30,
        )
        resp.raise_for_status()
        payload = resp.json()

        fixtures.extend(payload.get("data") or [])

        pagination = ((payload.get("meta") or {}).get("pagination")) or {}
        cur = pagination.get("current_page")
        tot = pagination.get("total_pages")
        if cur is None or tot is None or int(cur) >= int(tot):
            break
        page += 1

    fdf = pd.DataFrame(
        [
            {
                "FixtureID": f.get("id"),
                "Fixture": f.get("name") or str(f.get("id")),
                "StartingAt": f.get("starting_at"),
            }
            for f in fixtures
        ]
    )

    if fdf.empty:
        return fdf

    fdf["StartingAt"] = pd.to_datetime(fdf["StartingAt"], utc=True, errors="coerce")
    fdf = fdf.dropna(subset=["FixtureID", "StartingAt"])
    fdf = fdf[fdf["StartingAt"].between(now_utc, end_utc)].copy()
    return fdf.reset_index(drop=True)


def fetch_odds_for_fixture(
    fixture_id: int,
    session: requests.Session,
    market_ids: list[int] | None = None,
    bookmaker_ids: list[int] | None = None,
) -> list[dict]:
    """
    Gets odds from odds endpoints that include `total` and/or `handicap`.
    Tries pre-match first, then inplay as a fallback.

    If market_ids is provided, it will call .../markets/{market_id} for each.
    Otherwise it tries .../fixtures/{fixture_id} (all markets) and falls back if unsupported.
    """
    base_params = {"api_token": API_TOKEN, "include": ODDS_INCLUDE}

    # Optional filters (some endpoints accept filters; harmless if ignored)
    if bookmaker_ids:
        base_params["filters"] = f"bookmakers:{','.join(map(str, bookmaker_ids))}"

    endpoints_to_try = []

    if market_ids:
        for mid in market_ids:
            endpoints_to_try.append(f"/odds/pre-match/fixtures/{fixture_id}/markets/{mid}")
            endpoints_to_try.append(f"/odds/inplay/fixtures/{fixture_id}/markets/{mid}")
    else:
        endpoints_to_try.append(f"/odds/pre-match/fixtures/{fixture_id}")
        endpoints_to_try.append(f"/odds/inplay/fixtures/{fixture_id}")

    for ep in endpoints_to_try:
        r = session.get(f"{BASE_URL}{ep}", params=base_params, timeout=30)
        if r.status_code == 404:
            continue
        r.raise_for_status()
        data = r.json().get("data") or []
        rows = []

        for odd in data:
            market_obj = odd.get("market") or {}
            bookmaker_obj = odd.get("bookmaker") or {}

            player_ids = _extract_player_ids(odd)

            rows.append(
                {
                    "FixtureID": fixture_id,
                    "OddID": odd.get("id"),

                    "MarketID": odd.get("market_id"),
                    "Market": odd.get("market_description") or market_obj.get("name"),

                    # ✅ Correct line sources for these endpoints
                    "Line": _extract_line_from_odd(odd),

                    "Label": odd.get("label"),
                    "Odds": odd.get("value"),

                    "BookmakerID": odd.get("bookmaker_id"),
                    "Bookmaker": bookmaker_obj.get("name"),

                    # Player info (may be empty for non-player markets)
                    "PlayerIDs": player_ids,  # list[int]
                }
            )

        return rows

    return []


def fetch_odds_next_12h(
    market_ids: list[int] | None = None,
    bookmaker_ids: list[int] | None = None,
    resolve_player_names: bool = True,
) -> pd.DataFrame:
    """
    Combined approach:
      1) fetch fixtures in next 12h
      2) fetch odds per fixture from /odds/... endpoints to get proper Line (total/handicap)
      3) optionally resolve player IDs to names
    """
    fixtures_df = fetch_fixtures_next_12h()
    if fixtures_df.empty:
        return pd.DataFrame()

    session = requests.Session()
    rows = []

    for fid in fixtures_df["FixtureID"].astype(int).tolist():
        rows.extend(fetch_odds_for_fixture(fid, session, market_ids=market_ids, bookmaker_ids=bookmaker_ids))

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # Types
    df["Odds"] = pd.to_numeric(df["Odds"], errors="coerce")
    df["Line"] = pd.to_numeric(df["Line"], errors="coerce")  # numeric where possible

    # Join fixture metadata
    df = df.merge(fixtures_df, on="FixtureID", how="left")

    # Resolve player names (optional; uses caching)
    if resolve_player_names:
        cache: dict[int, str] = {}
        # expand to one row per player when there are multiple (rare)
        df = df.explode("PlayerIDs", ignore_index=True)
        df["PlayerID"] = pd.to_numeric(df["PlayerIDs"], errors="coerce").astype("Int64")
        df = df.drop(columns=["PlayerIDs"])

        def _name_or_none(pid):
            if pd.isna(pid):
                return None
            return fetch_player_name(int(pid), session, cache)

        df["PlayerName"] = df["PlayerID"].apply(_name_or_none)
    else:
        # keep list column
        pass

    # Drop unusable odds rows
    df = df.dropna(subset=["Odds", "MarketID", "Label", "StartingAt"])
    df = df.reset_index(drop=True)
    return df


# -------------------------
# Example usage
# -------------------------
df = fetch_odds_next_12h(resolve_player_names=True)

# Show relevant columns
cols = [
    "Fixture",
    "StartingAt",
    "MarketID",
    "Market",
    "Line",
    "Label",
    "Bookmaker",
    "Odds",
    "PlayerID",
    "PlayerName",
]

df[cols].head(50)

,Fixture,StartingAt,MarketID,Market,Line,Label,Bookmaker,Odds,PlayerID,PlayerName
0,Pisa vs Como,2026-01-06 14:00:00+00:00,92,Team Goalscorer,NaN,First,bet365,34.00,<NA>,None
1,Pisa vs Como,2026-01-06 14:00:00+00:00,92,Team Goalscorer,NaN,Last,bet365,9.00,<NA>,None
2,Pisa vs Como,2026-01-06 14:00:00+00:00,92,Team Goalscorer,NaN,Last,bet365,34.00,<NA>,None
3,Pisa vs Como,2026-01-06 14:00:00+00:00,251,1st Goal Scorer,NaN,First,bet365,9.00,<NA>,None
4,Pisa vs Como,2026-01-06 14:00:00+00:00,251,1st Goal Scorer,NaN,First,bet365,34.00,<NA>,None
5,Pisa vs Como,2026-01-06 14:00:00+00:00,285,Team Shots,NaN,1,bet365,2.00,<NA>,None
6,Pisa vs Como,2026-01-06 14:00:00+00:00,267,Player Shots On Target,NaN,1.5,bet365,15.00,<NA>,None
7,Pisa vs Como,2026-01-06 14:00:00+00:00,57,Correct Score,NaN,1:7,1xbet,100.00,<NA>,None
8,Pisa vs Como,2026-01-06 14:00:00+00:00,28,Goals Over/Under 1st Half,2.5,Over,1xbet,8.50,<NA>,None
9,Pisa vs Como,2026-01-06 14:00:00+00:00,106,Alternative 1st Half Asian Handicap,0.0,1,bet365,3.00,<NA>,None
